In [1]:
from moccasin import setup_notebook
setup_notebook()

In [2]:
from moccasin.config import get_active_network
active_network = get_active_network()
print(active_network.name)

eth-forked


In [77]:
from typing import Tuple
from boa.contracts.abi.abi_contract import ABIContract
from moccasin.config import get_active_network
import boa

STARTING_ETH_BALANCE = int(1000e18)
STARTING_WETH_BALANCE = int(1e18)
STARTING_USDC_BALANCE = int(100e6) #USDC has 6 decimials 
def _add_eth_balance():
    boa.env.set_balance(boa.env.eoa, STARTING_ETH_BALANCE)
    
def _add_token_balance(usdc,weth):
    print(f"WETH STARTING {weth.balanceOf(boa.env.eoa)}")
    weth.deposit(value = STARTING_WETH_BALANCE)
    print(f"WETH ENDING {weth.balanceOf(boa.env.eoa)}")
    
    
    #Our conncected Network address
    our_address = boa.env.eoa
    
    
    #Mint  USdc
    #We will need to prank ourselves as owners inorder to mint 
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)
    print(f"USDC ENDING {usdc.balanceOf(boa.env.eoa)}")
    usdc.configureMinter(our_address, STARTING_USDC_BALANCE )
    usdc.mint(our_address, STARTING_USDC_BALANCE)
    print(f"USDC ENDING {usdc.balanceOf(boa.env.eoa)}")
     
 
def setup_script() -> Tuple[ABIContract, ABIContract, ABIContract, ABIContract ]:
    print("Claiming faucets : ===============>")
    
    #Get active Network
    active_network = get_active_network()
    
    usdc = active_network.manifest_named("usdc")
    weth = active_network.manifest_named("weth")
    if active_network.is_local_or_forked_network():
        _add_eth_balance()
        _add_token_balance(usdc,weth)
    
    
    
def moccasin_main():
    setup_script()



moccasin_main()    

Claiming faucets : ===============>
WETH STARTING 1328620271631771072
WETH ENDING 2328620271631771072
USDC ENDING 4528489239
USDC ENDING 4628489239


In [4]:
    
usdc = active_network.manifest_named("usdc")
weth = active_network.manifest_named("weth")

In [65]:
usdc.balanceOf(boa.env.eoa)
weth.balanceOf(boa.env.eoa)

2000000000000000000

In [6]:
usdc

<usdc interface at 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48>

In [7]:
weth

<weth interface at 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2>

Deposit into AAVE

In [8]:
#Reload Config
from moccasin.config import get_or_initialize_config
config = get_or_initialize_config()
config.reload()
active_network = config.get_active_network()

aavev3_pool_address_provider = active_network.manifest_named("aavev3_pool_address_provider")
pool_address = aavev3_pool_address_provider.getPool()
print(pool_address)

config.reload()
active_network = config.get_active_network()
pool_contract = active_network.manifest_named("pool", address = pool_address )

0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2


In [9]:
pool_contract


<pool interface at 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2>

In [79]:
REFERRAL_CODE = 0

def deposit(pool_contract, token, amount):
    allowed_amount =token.allowance(boa.env.eoa, pool_contract)
    if allowed_amount  < amount:
        token.approve(pool_contract.address, amount)
    print(f"Deposting {token.name()} Money to AAVE Contract {pool_contract.address}")
    pool_contract.supply(token.address, amount, boa.env.eoa, REFERRAL_CODE)
    

usdc_balance = usdc.balanceOf(boa.env.eoa)
weth_balance = weth.balanceOf(boa.env.eoa)

if usdc_balance > 0:
    deposit(pool_contract,usdc, usdc_balance )
    
    
if weth_balance > 0:
    deposit(pool_contract,weth, weth_balance )
        

(
        totalCollateralBase,
        totalDebtBase,
        availableBorrowsBase,
        currentLiquidationThreshold,
        ltv,
        healthFactor,
    ) = pool_contract.getUserAccountData(boa.env.eoa)
print(f"""User account data:
    totalCollateralBase: {totalCollateralBase}
    totalDebtBase: {totalDebtBase}
    availableBorrowsBase: {availableBorrowsBase}
    currentLiquidationThreshold: {currentLiquidationThreshold}
    ltv: {ltv}
    healthFactor: {healthFactor}
        """)

Deposting USD Coin Money to AAVE Contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
Deposting Wrapped Ether Money to AAVE Contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
User account data:
    totalCollateralBase: 1343827942931
    totalDebtBase: 0
    availableBorrowsBase: 1049664006223
    currentLiquidationThreshold: 8082
    ltv: 7811
    healthFactor: 115792089237316195423570985008687907853269984665640564039457584007913129639935
        


# Get the List of AAVE tokens 

In [12]:
config.reload()
active_network = config.get_active_network()
aave_protocol_data_provider = active_network.manifest_named("aave_protocol_data_provider")
a_tokens = aave_protocol_data_provider.getAllATokens()
print(a_tokens)


[('aEthWETH', Address('0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8')), ('aEthwstETH', Address('0x0B925eD163218f6662a35e0f0371Ac234f9E9371')), ('aEthWBTC', Address('0x5Ee5bf7ae06D1Be5997A1A72006FE6C607eC6DE8')), ('aEthUSDC', Address('0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c')), ('aEthDAI', Address('0x018008bfb33d285247A21d44E50697654f754e63')), ('aEthLINK', Address('0x5E8C8A7243651DB1384C0dDfDbE39761E8e7E51a')), ('aEthAAVE', Address('0xA700b4eB416Be35b2911fd5Dee80678ff64fF6C9')), ('aEthcbETH', Address('0x977b6fc5dE62598B08C85AC8Cf2b745874E8b78c')), ('aEthUSDT', Address('0x23878914EFE38d27C4D67Ab83ed1b93A74D4086a')), ('aEthrETH', Address('0xCc9EE9483f662091a1de4795249E24aC0aC2630f')), ('aEthLUSD', Address('0x3Fe6a295459FAe07DF8A0ceCC36F37160FE86AA9')), ('aEthCRV', Address('0x7B95Ec873268a6BFC6427e7a28e396Db9D0ebc65')), ('aEthMKR', Address('0x8A458A9dc9048e005d22849F470891b840296619')), ('aEthSNX', Address('0xC7B4c17861357B8ABB91F25581E7263E08DCB59c')), ('aEthBAL', Address('0x2516E7B3F76

In [19]:
for a_token in a_tokens:
    if "WETH" in a_token[0]:
        a_weth = active_network.manifest_named("weth", address = a_token[1])
    if "USDC" in a_token[0]:
        a_usdc = active_network.manifest_named("usdc", address = a_token[1])
        
        
print(a_usdc)
print(a_weth)

<usdc interface at 0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c>
<weth interface at 0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8>


In [ ]:
# Get the amount in USDC and WETH deposited in AAVE

In [80]:
a_usdc_balance = a_usdc.balanceOf(boa.env.eoa)
a_weth_balance = a_weth.balanceOf(boa.env.eoa)

a_usdc_balance_normalized = a_usdc_balance / 1_000_000
a_weth_balance_normalized = a_weth_balance / 1_000_000_000_000_000_000

print(a_usdc_balance_normalized)
print(a_weth_balance_normalized)

5834.243715
2.328620271631771


# Get Price dunction

In [67]:
config.reload()

def get_price(feed_name:str):
    active_network = get_active_network()
    price_feed = active_network.manifest_named(feed_name)
    price = price_feed.latestAnswer()
    decimals = price_feed.decimals()
    decimal_normalized =  10 ** decimals
    return price / decimal_normalized


usdc_price = get_price("usdc_usd_price_feed")
weth_price = get_price("eth_usd_price_feed")


print(usdc_price)
print(weth_price)

1.00000731
3265.45


In [81]:
usdc_value = a_usdc_balance_normalized * usdc_price
weth_value = a_weth_balance_normalized * weth_price

total_value = usdc_value + weth_value

target_usdc_value = 0.3
target_weth_value = 0.7

usdc_percent_allocation = usdc_value /total_value
weth_percent_allocation  = weth_value  / total_value

BUFFER = 0.1


needs_rebalance =  (
    abs(usdc_percent_allocation - target_usdc_value) > BUFFER or abs(weth_percent_allocation - target_weth_value) > BUFFER
)

print(needs_rebalance)
print(usdc_percent_allocation)
print(weth_percent_allocation)

True
0.43415426759109443
0.5658457324089056


In [82]:
a_weth.approve(pool_contract.address, a_weth.balanceOf(boa.env.eoa))
print(a_weth.balanceOf(boa.env.eoa), boa.env.eoa,weth.address, "A WETH")
pool_contract.withdraw(weth.address, a_weth.balanceOf(boa.env.eoa), boa.env.eoa)
def print_token_balances():
    print(f"USDC BALANCE : ", {usdc.balanceOf(boa.env.eoa)})
    print(f"WETH BALANCE : ", {weth.balanceOf(boa.env.eoa)})
    print(f"aUSDC BALANCE : ", {a_usdc.balanceOf(boa.env.eoa)})
    print(f"aWETH BALANCE : ", {a_weth.balanceOf(boa.env.eoa)})
    
print_token_balances()

2328620271631771072 0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2 A WETH
USDC BALANCE :  {0}
WETH BALANCE :  {2328620271631771072}
aUSDC BALANCE :  {5834243715}
aWETH BALANCE :  {0}


# Calaculate rebalancing to ensure its weth is 70% of the portfolio and 30% of usdc

In [70]:
usdc_data = {"balance":a_usdc_balance_normalized, "price":usdc_price, "contract": usdc}
weth_data = {"balance":a_weth_balance_normalized, "price":weth_price, "contract": weth}

target_allocations = {"usdc": 0.3, "weth": 0.7}

def calculate_rebalancing_trades(
    usdc_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    weth_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    target_allocations: dict[str, float],  # {"usdc": 0.3, "weth": 0.7}
) -> dict[str, dict]:
    """
    Calculate the trades needed to rebalance a portfolio of USDC and WETH.

    Args:
        usdc_data: Dict containing USDC balance, price and contract
        weth_data: Dict containing WETH balance, price and contract
        target_allocations: Dict of token symbol to target allocation (must sum to 1)

    Returns:
        Dict of token symbol to dict containing contract and trade amount:
            {"usdc": {"contract": Contract, "trade": int},
             "weth": {"contract": Contract, "trade": int}}
    """
    # Calculate current values
    usdc_value = usdc_data["balance"] * usdc_data["price"]
    weth_value = weth_data["balance"] * weth_data["price"]
    total_value = usdc_value + weth_value

    # Calculate target values
    target_usdc_value = total_value * target_allocations["usdc"]
    target_weth_value = total_value * target_allocations["weth"]

    # Calculate trades needed in USD
    usdc_trade_usd = target_usdc_value - usdc_value
    weth_trade_usd = target_weth_value - weth_value

    # Convert to token amounts
    return {
        "usdc": {
            "contract": usdc_data["contract"],
            "trade": usdc_trade_usd / usdc_data["price"],
        },
        "weth": {
            "contract": weth_data["contract"],
            "trade": weth_trade_usd / weth_data["price"],
        },
    }

trades = calculate_rebalancing_trades(usdc_data, weth_data, target_allocations)
print(trades)
weth_to_sell = trades["weth"]["trade"]

{'usdc': {'contract': <usdc interface at 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48>, 'trade': 909.6278396204973}, 'weth': {'contract': <weth interface at 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2>, 'trade': -0.2785632880613715}}


# Swap collateral using uniswap router 

In [83]:
config.reload()

active_network = config.get_active_network()

uniswap_swap_router = active_network.manifest_named("uniswap_swap_router")
amount_weth = abs(int(weth_to_sell * (10 ** 18)))
print(amount_weth, "AMount to sell")

weth.approve(uniswap_swap_router.address, amount_weth)

print("SWAPPING : ======================>")
min_out = int(trades["usdc"]["trade"] * (10 ** 6) * 0.90)

print(min_out, "MININUM OUT")


    # struct ExactInputSingleParams {
    #     address tokenIn;
    #     address tokenOut;
    #     uint24 fee;
    #     address recipient;
    #     uint256 amountIn;
    #     uint256 amountOutMinimum;
    #     uint160 sqrtPriceLimitX96;
    # }

uniswap_swap_router.exactInputSingle(
    (
        weth.address,
        usdc.address,
        3000,
        boa.env.eoa,
        amount_weth,
        min_out,
        0
        
        
        
        
    )
    
)

print_token_balances()

278563288061371488 AMount to sell
SWAPPING : ======================>
818665055 MININUM OUT
USDC BALANCE :  {905641224}
WETH BALANCE :  {2050056983570399584}
aUSDC BALANCE :  {5834243715}
aWETH BALANCE :  {0}
